In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [8]:
import tensorflow as tf
from tensorflow import keras

### Importing Data

In [12]:
train_dir=r'D:\Internshala Videos\Deep Learning\Internshala ipynb\IR Project Data\Data\cats_and_dogs\train'
test_dir=r'D:\Internshala Videos\Deep Learning\Internshala ipynb\IR Project Data\Data\cats_and_dogs\test'
validation_dir=r'D:\Internshala Videos\Deep Learning\Internshala ipynb\IR Project Data\Data\cats_and_dogs\validation'

### Data Preprocessing

 Read the Picture Files

Decode the JPEG content to RBG grids of pixels

Convert these into floating point tensors

Rescale the pixel values(0,255) to the [0,1] interval

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [14]:
#generating Batches of tensor image data

train_datagen=ImageDataGenerator(rescale=1./255)
test_datagen=ImageDataGenerator(rescale=1./255)

train_generator=train_datagen.flow_from_directory(
    train_dir,      # go to train data folder in the directory
    target_size=(150,150),   #Reshaping size into 150 x 150 pixels
    batch_size=20,           #Data Stores as 20 batches
    class_mode='binary'
)

validation_generator=test_datagen.flow_from_directory(
    validation_dir,   #go to validation folder in the directory
    target_size=(150,150),   #Reshaping size into 150 x 150 pixels
    batch_size=20,           #Data Stores as 20 batches
    class_mode='binary'
)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


### Model 

In [16]:
from tensorflow.keras import layers
from tensorflow.keras import models

In [17]:
model=models.Sequential()

 #layer 1
model.add(layers.Conv2D(32,(3,3),activation='relu',input_size=(150,150,3)))  #(filters,window,activation,input_size)
model.add(layers.MaxPooling2D((2,2)))


# layer2
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.MaxPool2D(2,2))


#layer 3
model.add(layers.Conv2D(128,(3,3),activation='relu'))
model.add(layers.MaxPool2D(2,2))

#layer 4
model.add(layers.Conv2D(128,(3,3),activation='relu'))
model.add(layers.MaxPool2D(2,2))


#layer 5

model.add(layers.Flatten())

#layer 6
model.add(layers.Dense(512,activation='relu'))   #(neruons,activation)

 #output layer

model.add(layers.Dense(1,activation='sigmoid'))

